# Prepare Eigenstrat files to run PCA with HO SNPs
Extract and Merge in relevant populations

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-a-16-94.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 32
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


# Helper Functions

In [2]:
def return_pops(df, string, col="clst", 
                output=False):
    """Return list of clusters that contain string."""
    df1 = df[df[col].str.contains(string)]
    if output:
        print(df1[col].value_counts())
    clsts = list(set(df1[col].values))
    print(f"Found #clsts labels containing {string}: {len(clsts)}")

    return clsts

def run_convertf(path_convertf = "./o2bin/convertf", parfile = "./parfiles/convertf.keep.par"):
    """Runs the Downsampling"""
    ! $path_convertf -p $parfile

# Load the .ind File

In [3]:
base_path = "/n/groups/reich/DAVID/V46/V46.1/v46.1_HO"
ind_path = base_path + ".ind"

df_ind = pd.read_csv(ind_path, delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]
print(f"Loaded {len(df_ind)} Individuals")

Loaded 34761 Individuals


In [4]:
return_pops(df_ind, "Spain_BA")

Found #clsts labels containing Spain_BA: 2


['Spain_BA.SG', 'Spain_BA']

In [14]:
df_ind[df_ind["clst"].str.contains("Egypt_Dyn")]

,iid,sex,clst
14454,I4369,F,Egypt_Dynastic_lc
17656,I4270_all,M,Egypt_Dynastic_all
17932,I4281_all,F,Egypt_Dynastic_Nubian_all
18244,I4270,M,Egypt_Dynastic
18248,I4281,F,Egypt_Dynastic_Nubian


# Definie what target populations to pull

### Ancients

In [15]:
pops = ["Algeria", "Morocco", "Tunisia", "Punic", "Phoenician", "Spain_Vandal", "Spain_LBA"
        "Sardinia", "Ibiza", "Israel_MLBA", "Israel_LBA", "Ashkelon", "Sicily", "Hellenistic",
        "Israel_IA", "Israel_EIA", "Israel_Persian", "Gibraltar",
        "Spain_EBA_Afric", "Spain_BellBeaker_oAfrican", "Spain_Greek",
        "Spain_Hellenistic", "Spain_IA", "Italy_Sardinia_C_oAfrican", 
        "Nigeria_IA", "Nigeria_Medieval", "Mallorca", 
        "Egypt_Hellenistic", "Egypt_Roman", "Egypt_Dynastic",
        "Greece_"]

exclude_strings = ["_lc", "contam"]

clsts = [return_pops(df_ind, string=pop, 
                     output=False) for pop in pops]

clsts = [inner for ls in clsts for inner in ls]

Found #clsts labels containing Algeria: 4
Found #clsts labels containing Morocco: 8
Found #clsts labels containing Tunisia: 6
Found #clsts labels containing Punic: 33
Found #clsts labels containing Phoenician: 3
Found #clsts labels containing Spain_Vandal: 4
Found #clsts labels containing Spain_LBASardinia: 0
Found #clsts labels containing Ibiza: 1
Found #clsts labels containing Israel_MLBA: 12
Found #clsts labels containing Israel_LBA: 2
Found #clsts labels containing Ashkelon: 4
Found #clsts labels containing Sicily: 73
Found #clsts labels containing Hellenistic: 32
Found #clsts labels containing Israel_IA: 3
Found #clsts labels containing Israel_EIA: 1
Found #clsts labels containing Israel_Persian: 1
Found #clsts labels containing Gibraltar: 2
Found #clsts labels containing Spain_EBA_Afric: 3
Found #clsts labels containing Spain_BellBeaker_oAfrican: 2
Found #clsts labels containing Spain_Greek: 3
Found #clsts labels containing Spain_Hellenistic: 4
Found #clsts labels containing Spai

### Moderns
Get list of Human Origin Populations to keep

In [16]:
path_ho = "/n/groups/reich/hringbauer/git/punic_aDNA/parfiles/pca/construct_WE_NA_PCA.list"

df_ho = pd.read_csv(path_ho, header=None, sep=r"\s+", engine="python")
df_ho.columns=["iid", "pop"]
print(f"Loaded {len(df_ho)} Individuals")

pops = set(df_ho["pop"])
clsts1 = [p.rsplit("_", 1)[0] for p in pops]
l = [np.sum(df_ind["clst"].str.contains(p)) for p in clsts1]
assert(np.min(l)>0)

Loaded 1196 Individuals


# Prepare and save final pop list

In [19]:
exclude_strings = ["_lc", "contam", "_d"]

clsts = list(set(clsts).union(set(clsts1))) # Filter to unique Elements
print(f"Loaded {len(clsts)} Populations")

### Exclude Strings
for ex in exclude_strings:
    clsts = [c for c in clsts if ex not in c]
print(f"After Exclusion {len(clsts)} populations")

### Originally Loaded 379 Populations
# After Exclusion 289 populations

Loaded 268 Populations
After Exclusion 268 populations


In [20]:
keep = np.array(clsts)
path_keep = "./parfiles/pca/keep_pops"
np.savetxt(path_keep, keep, fmt="%s")
print(f"Saved {len(keep)} population names to {path_keep}")

Saved 268 population names to ./parfiles/pca/keep_pops


# Run convertf
Takes about 20 min for all individuals

Check additional parameters in manually encoded parfile!!

In [21]:
%%time
run_convertf(path_convertf = "/n/groups/reich/hringbauer/o2bin/convertf", 
             parfile = "./parfiles/pca/convertf.keep.par")

parameter file: ./parfiles/pca/convertf.keep.par
BASE: /n/groups/reich/
DIR: DAVID/V46/V46.1/v46.1_HO
OUT: hringbauer/git/punic_aDNA/eigenstrat/punic1.v46_HO
genotypename: /n/groups/reich//DAVID/V46/V46.1/v46.1_HO.geno
snpname: /n/groups/reich//DAVID/V46/V46.1/v46.1_HO.snp
indivname: /n/groups/reich//DAVID/V46/V46.1/v46.1_HO.ind
genooutfilename: /n/groups/reich//hringbauer/git/punic_aDNA/eigenstrat/punic1.v46_HO.geno
snpoutfilename: /n/groups/reich//hringbauer/git/punic_aDNA/eigenstrat/punic1.v46_HO.snp
indoutfilename: /n/groups/reich//hringbauer/git/punic_aDNA/eigenstrat/punic1.v46_HO.ind
outputformat: PACKEDANCESTRYMAP
hashcheck: YES
poplistname: /n/groups/reich//hringbauer/git/punic_aDNA/parfiles/pca/keep_pops
## /n/groups/reich/hringbauer/o2bin/convertf version: 5722
read 1073741824 bytes
read 2147483648 bytes
read 3221225472 bytes
read 4294967296 bytes
read 5193506943 bytes
packed geno read OK
end of inpack
before compress: snps: 597573 indivs: 34761
after compress: snps: 597573 i

# Modify the .ind file to have one population to project on in moderns

In [ ]:
path_ind = "/n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v46_HO.ind"
path_mod = "/n/groups/reich/hringbauer/git/punic_aDNA/eigenstrat/punic1.v46_HO_mod.ind"

df_ind = pd.read_csv(path_ind, header=None, sep=r"\s+", engine="python")
df_ind.columns = ["iid", "sex", "pop"]
iids = df_ho["iid"].values # Alissas original IIDs
idx = [iid in iids for iid in df_ind["iid"]]
print(f"Found {np.sum(idx)}/{len(idx)} HO individuals")

df_ind.loc[idx, "pop"]  = "construct_WE_NA_PCA" #df_ind.loc[idx, "pop"] + "_mod" 
df_ind.to_csv(path_mod, sep=" ", index=None, header=False)
print(f"Saved {len(df_ind)} overall individuals to {path_mod}")

### Sanity Check 
#idx = [p in pops for p in df_ind["pop"]] 
idx = [(p=="construct_WE_NA_PCA") for p in df_ind["pop"]] 
print(f"Found {np.sum(idx)}/{len(idx)} of Alissas _mod pops")
# in v45: 1196/2169 

In [33]:
### Needed only for trouble shooting ###
found = [iid in df_ind["iid"].values for iid in df_ho["iid"]]
print(f"Found {np.sum(found)}/{len(found)} of Alissas _mod pops")
#df_ho[~np.array(found)]["pop"].value_counts() # Only for

Found 1196/1196 of Alissas _mod pops


In [ ]:
df_ind["pop"].value_counts()

In [ ]:
df_ind[df_ind["pop"].str.contains("Tunisia_Ph")]

# And now sbatch the PCA script.
Takes about 9h for 1000 extra samples

See in `./parfiles/pca/`

# Area 51

In [51]:
path_ho = "/n/groups/reich//hringbauer/git/punic_aDNA/eigenstrat/punic1.v44_HO.ind"

df_ho = pd.read_csv(path_ho, header=None, sep=r"\s+", engine="python")
#df_ho.columns=["iid", "pop"]

In [52]:
df_ho

,0,1,2
0,BBR2,M,Ignore_Morocco_Berber(first_degree_relative)
1,BBR6,M,Ignore_Morocco_Berber(first_degree_relative)
2,BBR18,M,Ignore_Morocco_Berber(first_degree_relative)
3,BBR21,M,Ignore_Morocco_Berber(first_degree_relative)
4,BBR23,M,Ignore_Morocco_Berber(first_degree_relative)
...,...,...,...
1940,I21964_d,U,Italy_Sardinia_Punic
1941,I22093_d,U,Italy_Sardinia_Punic
1942,I21971_d,U,Italy_Sicily_Punic
1943,I22094_d,U,Italy_Sardinia_Punic
